In [1]:
import time
import numpy as np
import pandas as pd
import json
import seaborn as sb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sb
import pickle

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
dbname = 'tutor_db'
username = 'codonnell'
pswd = '7883511'

con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# query:
sql_query = """
SELECT * FROM tutor_main
"""
tutor_data = pd.read_sql_query(sql_query,con)

In [5]:
test_data = tutor_data[(tutor_data['hourly_rate']<100) 
                               & (tutor_data['hourly_rate']>20)]

In [13]:
#len(test_data[test_data['inst_1'].str.contains('State')])
test_data['inst_1'].value_counts()

{}                                          296
Arizona State                                72
UCLA                                         58
San Diego State                              54
Boston                                       52
                                           ... 
the Sciences                                  1
Alabama  Huntsville                           1
Louisiana State , Baton Rouge, LA             1
Current: Montgomery : Takoma Park Campus      1
St. Scholastica                               1
Name: inst_1, Length: 3470, dtype: int64

In [15]:
edges = [20,40,60,80,100]
edu_bins = []

state_cols1 = []
state_cols2 = []
state_cols3 = []

for i in range(0,len(edges)-1):
    df = test_data[(test_data['hourly_rate']>=edges[i])& (test_data['hourly_rate']<edges[i+1])]
    state_cols1.append(len(df[df['inst_1'].str.contains('State')])/len(df))
    state_cols2.append(len(df[df['inst_2'].str.contains('State')])/len(df))
    state_cols3.append(len(df[df['inst_3'].str.contains('State')])/len(df))
    #print(df['inst_1'].value_counts()
print(state_cols1)
print(state_cols2)
print(state_cols3)

[0.14813432835820894, 0.12457555178268251, 0.11637931034482758, 0.11984282907662082]
[0.07201492537313432, 0.08234295415959253, 0.078125, 0.08055009823182711]
[0.014552238805970149, 0.01803904923599321, 0.023168103448275863, 0.023575638506876228]


In [21]:
ivys = 'Brown|Columbia|Dartmouth|Harvard|Cornell|Pennsylvania|Princeton|Yale'
ivy_cols1 = []
ivy_cols2 = []
ivy_cols3 = []

for i in range(0,len(edges)-1):
    df = test_data[(test_data['hourly_rate']>=edges[i])& (test_data['hourly_rate']<edges[i+1])]
    ivy_cols1.append(len(df[df['inst_1'].str.contains(ivys)])/len(df))
    ivy_cols2.append(len(df[df['inst_2'].str.contains(ivys)])/len(df))
    ivy_cols3.append(len(df[df['inst_3'].str.contains(ivys)])/len(df))
    #print(df['inst_1'].value_counts()
print(ivy_cols1)
print(ivy_cols2)
print(ivy_cols3)

[0.027238805970149254, 0.03416808149405773, 0.05118534482758621, 0.08840864440078586]
[0.017164179104477612, 0.022071307300509338, 0.03987068965517242, 0.07269155206286837]
[0.004477611940298508, 0.010186757215619695, 0.014547413793103448, 0.025540275049115914]


In [24]:
len(test_data[test_data['inst_3'].str.contains(ivys)])

100

In [52]:
all_edu_lists = [i for i in sample['education_list'].values]
all_edus = [item for sublist in all_edu_lists for item in sublist]
edu_counts = pd.Series(all_edus, name = 'edu_counts').value_counts()

In [54]:
edu_counts[5:20]

and            1850
California     1740
PhD            1467
Columbia       1421
New            1411
undergrad      1353
Graduate       1338
Mathematics    1282
School         1275
Engineering    1166
Coursework     1152
York           1150
Science        1143
English        1140
Physics        1101
Name: edu_counts, dtype: int64

In [57]:
edu_bigtext = listToString([i for i in sample['education_clean'].values])

In [59]:
import collections
def ngrams(text, n=2):
    return zip(*[text[i:] for i in range(n)])
ngram_counts_2 = collections.Counter(ngrams(edu_bigtext.split(), 2))
ngram_counts_2.most_common(20)

[(('University', 'of'), 5745),
 (('State', 'University'), 1728),
 (('University', 'Masters'), 1720),
 (('Columbia', 'University'), 1158),
 (('Graduate', 'Coursework'), 1152),
 (('College', 'of'), 1105),
 (('New', 'York'), 1104),
 (('Masters', 'University'), 1086),
 (('of', 'California'), 911),
 (('School', 'of'), 801),
 (('of', 'Technology'), 687),
 (('Institute', 'of'), 623),
 (('City', 'College'), 536),
 (('of', 'New'), 532),
 (('University', 'English'), 504),
 (('University', 'PhD'), 473),
 (('York', 'University'), 455),
 (('Computer', 'Science'), 450),
 (('California', 'State'), 389),
 (('of', 'Texas'), 367)]

In [60]:
def ngrams(text, n=3):
    return zip(*[text[i:] for i in range(n)])
ngram_counts_3 = collections.Counter(ngrams(edu_bigtext.split(), 3))
ngram_counts_3.most_common(20)

[(('Masters', 'University', 'of'), 1072),
 (('University', 'of', 'California'), 876),
 (('Institute', 'of', 'Technology'), 514),
 (('of', 'New', 'York'), 468),
 (('City', 'College', 'of'), 456),
 (('New', 'York', 'University'), 413),
 (('State', 'University', 'Masters'), 365),
 (('University', 'of', 'Texas'), 354),
 (('College', 'of', 'New'), 343),
 (('The', 'University', 'of'), 341),
 (('California', 'State', 'University'), 325),
 (('Graduate', 'Coursework', 'University'), 278),
 (('Coursework', 'University', 'of'), 276),
 (('of', 'California', 'Berkeley'), 267),
 (('Masters', 'Columbia', 'University'), 259),
 (('University', 'Graduate', 'Coursework'), 259),
 (('University', 'Masters', 'University'), 258),
 (('PhD', 'University', 'of'), 257),
 (('University', 'of', 'Michigan'), 253),
 (('Columbia', 'University', 'Masters'), 240)]